# MNIST Basic Approach

In [35]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [19]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


### Helper Function

In [20]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

In [21]:
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

In [22]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

In [23]:
def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                          strides=[1, 2, 2, 1], padding='SAME')

In [24]:
def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W) + b)

In [36]:
def fully_connected_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

### Placeholder

In [37]:
X = tf.placeholder(tf.float32, shape=[None, 784])

In [38]:
Y_true = tf.placeholder(tf.float32, shape=[None, 10])

### Layers

In [39]:
X_image = tf.reshape(X, [-1, 28,28,1])

#### First Conv Layer

In [54]:
conv_1 = convolutional_layer(X_image, shape=[6,6,1,32])

In [55]:
conv_2_pooling = max_pool_2by2(conv_1)

#### Second Conv Layer

In [56]:
conv_2 = convolutional_layer(conv_2_pooling, shape=[6,6,32,64])

In [57]:
conv_2_pooling =  max_pool_2by2(conv_2)

#### Flattening Layer

In [59]:
flattening = tf.reshape(conv_2_pooling, [-1, 7*7*64])

#### FCL

In [60]:
fully_connected_layer_one = tf.nn.relu(fully_connected_layer(flattening, 1024))

#### Dropout

In [61]:
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(fully_connected_layer_one, keep_prob=hold_prob)

#### Output Layer

In [62]:
Y_pred = fully_connected_layer(full_one_dropout,10)

#### Loss Function

In [63]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y_true, logits=Y_pred))

#### Optimizer

In [64]:
optimizer = tf.train.AdamOptimizer(learning_rate=0.00001)
train = optimizer.minimize(cross_entropy)

#### Initialize Variables

In [65]:
init = tf.global_variables_initializer()

#### Session

In [76]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x , batch_y = mnist.train.next_batch(50)
        
        sess.run(train,feed_dict={X:batch_x,Y_true:batch_y,hold_prob:0.5})
        
        # PRINT OUT A MESSAGE EVERY 100 STEPS
        if i%100 == 0:
            
            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # Test the Train Model
            matches = tf.equal(tf.argmax(Y_pred,1),tf.argmax(Y_true,1))

            acc = tf.reduce_mean(tf.cast(matches,tf.float32))

            print(sess.run(acc,feed_dict={X:mnist.test.images,Y_true:mnist.test.labels,hold_prob:1.0}))
            print('\n')

Currently on step 0
Accuracy is:
0.114


Currently on step 100
Accuracy is:
0.3145


Currently on step 200
Accuracy is:
0.5742


Currently on step 300
Accuracy is:
0.6895


Currently on step 400
Accuracy is:
0.755


Currently on step 500
Accuracy is:
0.7934


Currently on step 600
Accuracy is:
0.8211


Currently on step 700
Accuracy is:
0.8412


Currently on step 800
Accuracy is:
0.8473


Currently on step 900
Accuracy is:
0.8673


Currently on step 1000
Accuracy is:
0.8753


Currently on step 1100
Accuracy is:
0.8842


Currently on step 1200
Accuracy is:
0.8886


Currently on step 1300
Accuracy is:
0.8939


Currently on step 1400
Accuracy is:
0.8949


Currently on step 1500
Accuracy is:
0.9026


Currently on step 1600
Accuracy is:
0.9055


Currently on step 1700
Accuracy is:
0.9106


Currently on step 1800
Accuracy is:
0.9125


Currently on step 1900
Accuracy is:
0.9165


Currently on step 2000
Accuracy is:
0.9188


Currently on step 2100
Accuracy is:
0.9208


Currently on step 2200
A

In [77]:
Y_true.get_shape()

TensorShape([Dimension(None), Dimension(10)])

In [78]:
len(mnist.test.labels)

10000